In [ ]:
# 데이터 전처리
# 1. yolov5 설치 (git clone https://github.com/ultralytics/yolov5.git)

In [5]:
import os
import json
import random

def convert_to_yolo_format(data_list, output_folder):
    for image_data in data_list:
        image_filename = image_data['file_name']
        bbox_data = image_data['bbox_data']

        image_path = os.path.join('./dataset/img', image_filename)

        txt_filename = os.path.splitext(image_filename)[0] + ".txt"
        txt_filepath = os.path.join(output_folder, txt_filename)

        count = 0
        with open(txt_filepath, 'w') as txtfile:
            for bbox_info in bbox_data:
                x_min, y_min, width, height = bbox_info['bbox']
                x_max = x_min + width
                y_max = y_min + height
                class_ids = [0,1,2,3]

                # Convert coordinates to YOLO format (normalized)
                img_width, img_height = 3000, 4000  # Replace with actual image dimensions
                x_center = (x_min + x_max) / (2.0 * img_width)
                y_center = (y_min + y_max) / (2.0 * img_height)
                w = width / img_width
                h = height / img_height
                txtfile.write(f"{class_ids[count]} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
                count += 1

def main():
    # 폴더 경로 설정
    folder_path = './dataset/lab'

    # 새로운 파일 경로와 이름
    new_file_path = './dataset/etc'

    # 모든 데이터를 담을 리스트
    all_data = []

    # 폴더 내의 모든 파일 목록 가져오기
    file_list = os.listdir(folder_path)

    # JSON 파일 목록 필터링

    # 파일 개수 확인
    file_count = len(file_list)
    print(file_count)

    # JSON 파일들을 순회하면서 파일 이름과 bbox 데이터 추출
    for json_file in file_list:
        json_path = os.path.join(folder_path, json_file)

        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            images = data.get('images', [])
            annotations = data.get('annotations', [])

            # 파일 이름과 bbox 데이터 저장
            for image in images:
                file_name = image.get('file_name', None)
                image_id = image.get('id', None)

                # 해당 이미지의 머리, 몸통, 꼬리 bbox 추출
                bbox_data = []
                for annotation in annotations:
                    if annotation.get('image_id', None) == image_id:
                        category_id = annotation.get('category_id_region', None)
                        bbox = annotation.get('bbox', None)
                        bbox_data.append({'category_id': category_id, 'bbox': bbox})

                data_entry = {
                    'file_name': file_name,
                    'bbox_data': bbox_data
                }
                all_data.append(data_entry)

    # 새로운 폴더가 없는 경우 생성
    os.makedirs(os.path.dirname(new_file_path), exist_ok=True)

    # 데이터를 랜덤하게 섞어서 훈련 데이터와 검증 데이터로 분할
    random.shuffle(all_data)
    train_data = all_data[:int(0.8 * len(all_data))]
    val_data = all_data[int(0.8 * len(all_data)):]

    # 훈련 데이터와 검증 데이터를 각각 YOLO txt 파일로 저장
    train_txt_path = os.path.join(new_file_path, 'train')
    val_txt_path = os.path.join(new_file_path, 'val')

    os.makedirs(train_txt_path, exist_ok=True)
    os.makedirs(val_txt_path, exist_ok=True)

    convert_to_yolo_format(train_data, train_txt_path)
    convert_to_yolo_format(val_data, val_txt_path)

    print('YOLOv5용 txt 파일을 생성하였습니다.')

if __name__ == "__main__":
    main()


6965
YOLOv5용 txt 파일을 생성하였습니다.


In [4]:
import os
import json
import random

def load_json_data(folder_path):
    all_data = []
    file_list = os.listdir(folder_path)

    for json_file in file_list:
        json_path = os.path.join(folder_path, json_file)

        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            images = data.get('images', [])
            annotations = data.get('annotations', [])

            for image in images:
                file_name = image.get('file_name', None)
                image_id = image.get('id', None)

                bbox_data = []
                for annotation in annotations:
                    if annotation.get('image_id', None) == image_id:
                        category_id = annotation.get('category_id_region', None)
                        bbox = annotation.get('bbox', None)
                        bbox_data.append({'category_id': category_id, 'bbox': bbox})

                data_entry = {
                    'file_name': file_name,
                    'bbox_data': bbox_data
                }
                all_data.append(data_entry)

    return all_data

def split_data(all_data):
    random.shuffle(all_data)
    train_data = all_data[:int(0.8 * len(all_data))]
    val_data = all_data[int(0.8 * len(all_data)):]

    return train_data, val_data

def create_txt_file(data, txt_file_path):
    img_path = '../dataset/img/'
    with open(txt_file_path, 'w') as txt_file:
        for item in data:
            txt_file.write(img_path + item['file_name'] + '\n')
            
            count = 0
            # 추가: bbox 정보를 가져와서 작성
            for bbox_info in item['bbox_data']:
                # class_id = bbox_info['category_id']
                class_ids = [0,1,2,3]
                x_min, y_min, width, height = bbox_info['bbox']
                
                x_max = x_min + width
                y_max = y_min + height
                
                # Convert bbox coordinates to YOLO format (normalized)
                img_width, img_height = 3000, 4000  # Replace with actual image dimensions
                x_center = (x_min + x_max) / (2.0 * img_width)
                y_center = (y_min + y_max) / (2.0 * img_height)
                w = width / img_width
                h = height / img_height

                # Write bbox info to txt file
                txt_file.write(f" {x_center:.6f},{y_center:.6f},{w:.6f},{h:.6f},{class_ids[count]}")
                count += 1

            txt_file.write('\n')

def main():
    folder_path = './dataset/lab'
    new_file_path = './dataset/etc'

    all_data = load_json_data(folder_path)
    train_data, val_data = split_data(all_data)

    os.makedirs(new_file_path, exist_ok=True)

    train_txt_path = os.path.join(new_file_path, 'train.txt')
    val_txt_path = os.path.join(new_file_path, 'val.txt')

    create_txt_file(train_data, train_txt_path)
    create_txt_file(val_data, val_txt_path)

    print('train.txt and val.txt files created.')

if __name__ == "__main__":
    main()


train.txt and val.txt files created.


In [2]:
from ultralytics import YOLO
import cv2
import os
import urllib

model = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:42<00:00, 528kB/s] 


In [ ]:
# python val.py --img 640 --batch 16 --data data.yaml --weights runs/train/my_custom_model/weights/best.pt --task test
# python train.py --batch 32 --epochs 10 --data data.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt
# python train.py --batch 32 --epochs 5 --data data.yaml --cfg models/yolov5s.yaml --weights yolov8s.pt

# python train.py --img 640 --batch 16 --epochs 1 --data data.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --device 0


In [1]:
from tensorflow.python.client import device_lib

local_devices = device_lib.list_local_devices()

# 모든 장치 정보 출력
print(local_devices)

# GPU 정보만 출력
# gpu_devices = [x for x in local_devices if 'GPU' in x.device_type]
# print(gpu_devices)

# CPU 정보만 출력
# cpu_devices = [x for x in local_devices if 'CPU' in x.device_type]
# print(cpu_devices)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2730868882631989703
xla_global_id: -1
]


In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18242263868752044040
 xla_global_id: -1]